---
jupyter: python3
from: markdown+emoji
execute:
  echo: true
  eval: false
  message: false
  warning: false
---

# Classifications d'images supervisées {#sec-chap05}


## :rocket: Préambule

Assurez-vous de lire ce préambule avant d'exécutez le reste du notebook.

### :dart: Objectifs
Dans ce chapitre, nous abordons quelques techniques de traitement d'images dans le domaine spatial uniquement. Ce chapitre est aussi disponible sous la forme d'un notebook Python sur Google Colab:

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sfoucher/TraitementImagesPythonVol1/blob/main/notebooks/04-TransformationSpatiales.ipynb){target="_blank"} 

### Librairies
Les librairies qui vont être explorées dans ce chapitre sont les suivantes:

* [SciPy -](https://scipy.org/)

* [NumPy -](https://numpy.org/) 

* [opencv-python · PyPI](https://pypi.org/project/opencv-python/)

* [scikit-image](https://scikit-image.org/)

* [Rasterio](https://rasterio.readthedocs.io/en/stable/)

* [Xarray](https://docs.xarray.dev/en/stable/)

* [rioxarray](https://corteva.github.io/rioxarray/stable/index.html)

Dans l'environnement Google Colab, seul `rioxarray` doit être installés:

In [ ]:
%%capture
!pip install -qU matplotlib rioxarray xrscipy

Vérifier les importations:

In [ ]:
#| eval: true
import numpy as np
import rioxarray as rxr
from scipy import signal
import xarray as xr
import xrscipy
import matplotlib.pyplot as plt

### Images utilisées

Nous allons utilisez les images suivantes dans ce chapitre:

In [ ]:
%%capture
!wget https://github.com/sfoucher/TraitementImagesPythonVol1/raw/refs/heads/main/data/chapitre01/subset_RGBNIR_of_S2A_MSIL2A_20240625T153941_N0510_R011_T18TYR_20240625T221903.tif -O RGBNIR_of_S2A.tif
!wget https://github.com/sfoucher/opengeos-data/raw/refs/heads/main/raster/landsat7.tif -O landsat7.tif
!wget https://github.com/sfoucher/opengeos-data/raw/refs/heads/main/images/berkeley.jpg -O berkeley.jpg
!wget https://github.com/sfoucher/TraitementImagesPythonVol1/raw/refs/heads/main/data/chapitre01/subset_0_of_S1A_split_NR_Cal_Deb_ML_Spk_SRGR.tif -O SAR.tif

Vérifiez que vous êtes capable de les lire :

In [ ]:
#| eval: true
#| output: false

with rxr.open_rasterio('berkeley.jpg', mask_and_scale= True) as img_rgb:
    print(img_rgb)
with rxr.open_rasterio('RGBNIR_of_S2A.tif', mask_and_scale= True) as img_rgbnir:
    print(img_rgbnir)
with rxr.open_rasterio('SAR.tif', mask_and_scale= True) as img_SAR:
    print(img_SAR)

## Classification d'images pixel par pixel {#sec-051}

### Parallélépipède {#sec-0511}

### Méthodes paramétriques {#sec-0512}

### Méthodes non paramétriques {#sec-0513}

### SVEM, réseaux de neurones, forêts aléatoires {#sec-0514}

## Segmentation d’images {#sec-052}

### Classification objet {#sec-0521}

### Approches par arbre (BPT, etc.) {#sec-0522}


## Quiz de révision du chapitre {#sec-053}


## Exercices de révision {#sec-054}